In [1]:
import torch 
from torch.utils.data import Dataset,Subset, DataLoader, TensorDataset, ConcatDataset
import torchvision
import os
from PIL import Image, ImageFile
from torchvision import transforms, datasets
from pathlib import Path
# split validation set into new train and validation set
from sklearn.model_selection import train_test_split
#plot examples
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import torch.nn as nn
torch.manual_seed(42)
np.random.seed(42)
import copy

from baselineCNN import *

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
dataset_path = Path('./../wildfire-prediction-dataset')


In [3]:
pretrain_path = dataset_path / 'train'
val_path = dataset_path / 'valid'
test_path = dataset_path / 'test'

In [4]:
dataset = datasets.ImageFolder(test_path, transform=transforms.ToTensor())

In [5]:
def get_all_datasets(pretrain_path, val_path, test_path, transforms):
    
    pretrain_dataset = datasets.ImageFolder(pretrain_path, transform=transforms['pretrain'])
    val_dataset = datasets.ImageFolder(val_path, transform=transforms['valid'])
    test_dataset = datasets.ImageFolder(test_path, transform=transforms['test'])
    train_idx, validation_idx = train_test_split(np.arange(len(val_dataset)),
                                             test_size=0.2,
                                             random_state=42,
                                             shuffle=True,
                                             stratify=val_dataset.targets)
    train_dataset = Subset(val_dataset, train_idx)
    val_dataset = Subset(val_dataset, validation_idx)
    
    return pretrain_dataset, train_dataset, val_dataset, test_dataset

In [6]:
num_epochs = 10
batch_size = 32  


# Data transformations
data_transforms = {
    'pretrain': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'valid': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
    ]),
}
unlabeled, train_dataset, val_dataset, test_dataset = get_all_datasets(pretrain_path, val_path, test_path, data_transforms)

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=6)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=6)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=6)

unlabeled_loader = DataLoader(unlabeled, batch_size=batch_size, shuffle=True, num_workers=6)

In [7]:
def validate(model, data_loader, loss_fn, device):
    model.eval()
    losses = []
    correct_predictions = 0
    with torch.no_grad():
        for x, y in tqdm(data_loader):
            x = x.to(device).half()  # Convert to float16
            y = y.to(device)
            with torch.amp.autocast('cuda'):
                y_hat = model(x)
                loss = loss_fn(y_hat, y)
            losses.append(loss.item())
            correct_predictions += (y == y_hat.argmax(1)).sum().item()
    return losses, correct_predictions

def train_one_epoch(model, optimizer, data_loader, loss_fn, device):
    model.train()
    losses = []
    for x, y in tqdm(data_loader):
        x = x.float().to(device).half()  # Convert to float16
        y = y.to(device)
        optimizer.zero_grad()
        with torch.amp.autocast('cuda'):  # Use automatic mixed precision
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return losses

def train_model(model, num_epochs, optimizer, train_loader, val_loader, criterion, device, best_model_path):
    model.train()
    best_val_accuracy = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        train_loss = train_one_epoch(model, optimizer, train_loader, criterion, device)
        val_loss, correct_predictions = validate(model, val_loader, criterion, device)
        val_accuracy = correct_predictions / len(val_dataset)

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_accuracy': val_accuracy,
            }, best_model_path)
        
        print(f'Train Loss: {np.mean(train_loss):.4f} Validation Loss: {np.mean(val_loss):.4f} Validation Accuracy: {val_accuracy:.4f}')
    return model


def pseudo_label_dataset(model, unlabeled_loader, device, confidence_threshold=0.95):
    model.eval()
    pseudo_inputs = []
    pseudo_labels = []
    
    with torch.no_grad():
        for inputs, _ in tqdm(unlabeled_loader, desc="Generating pseudo-labels"):
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = torch.softmax(outputs, dim=1)
            max_probs, preds = torch.max(probabilities, 1)
            
            # Select samples with high confidence
            confident_mask = max_probs >= confidence_threshold
            if confident_mask.sum() > 0:
                confident_inputs = inputs[confident_mask].cpu()
                confident_labels = preds[confident_mask].cpu()
                
                pseudo_inputs.append(confident_inputs)
                pseudo_labels.append(confident_labels)
    
    if not pseudo_inputs:
        return None
    
    # Combine all selected samples
    pseudo_inputs = torch.cat(pseudo_inputs, 0)
    pseudo_labels = torch.cat(pseudo_labels, 0)
    
    pseudo_dataset = TensorDataset(pseudo_inputs, pseudo_labels.long())
    print(f"Generated {len(pseudo_dataset)} pseudo-labeled samples from {len(unlabeled_loader.dataset)} with a confidence threshold = {confidence_threshold}")
    return pseudo_dataset

def custom_collate(batch):
    images, labels = zip(*batch)
    images = torch.stack(images, 0)
    # Convert all labels to a tensor
    labels = torch.tensor(labels)
    return images, labels

In [8]:
baseline = BaselineModel()

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
baseline = baseline.to(device)
checkpoint = torch.load("baseline.pth")
baseline.load_state_dict(checkpoint['model_state_dict'])

/tmp/ipykernel_3832431/1246028678.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("baseline.pth")


<All keys matched successfully>

In [10]:
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
# Test the model
baseline.eval()
test_loss, correct_predictions = validate(baseline, test_data_loader, criterion, device)
base_accuracy = correct_predictions / len(test_dataset)
print(f'Test Loss: {np.mean(test_loss):.4f} Test Accuracy: {base_accuracy:.4f}')

100%|██████████| 197/197 [00:15<00:00, 13.00it/s]

Test Loss: 0.1906 Test Accuracy: 0.9417


In [12]:
confidence_threshold = 0.95  # Initial confidence threshold
initial_acc = base_accuracy
num_iterations = 3
num_epochs = 10

In [13]:
# unlabeled = Subset(unlabeled, list(range(10)))
# train_dataset = Subset(train_dataset, list(range(10)))

In [ ]:

model = baseline

# Keep track of original labeled dataset
original_train_dataset = train_dataset

# Keep track of unlabeled samples and which ones have been pseudo-labeled
remaining_unlabeled = list(range(len(unlabeled)))
all_pseudo_labeled_indices = set()
all_pseudo_labeled_datasets = []


for iteration in range(num_iterations):
    print("-"*100)
    print(f"\nPseudo-labeling iteration :  {iteration+1}/{num_iterations} \n ")
    
    # Create a loader only for remaining unlabeled data
    remaining_unlabeled_dataset = torch.utils.data.Subset(unlabeled, remaining_unlabeled)
    unlabeled_loader = DataLoader(remaining_unlabeled_dataset, batch_size=batch_size)
    
    # Generate pseudo-labels for the remaining unlabeled data
    pseudo_dataset = pseudo_label_dataset(model, unlabeled_loader, device, confidence_threshold)
    

    if pseudo_dataset is None or len(pseudo_dataset) == 0:
        print(f"No confident samples found at threshold {confidence_threshold}. Lowering threshold.")
        confidence_threshold *= 0.9  # Reduce threshold

    else : 
        # Store this iteration's pseudo-labeled dataset
        all_pseudo_labeled_datasets.append(pseudo_dataset)

        # Remove pseudo-labeled indices from remaining_unlabeled
        # We need to track which indices from the original dataset were used
        pseudo_indices = []
        batch_idx = 0
        for data, _ in unlabeled_loader:
            outputs = model(data.to(device))
            probabilities = torch.softmax(outputs, dim=1)
            max_probs, _ = torch.max(probabilities, 1)
            confident_mask = max_probs >= confidence_threshold
            
            for j, is_confident in enumerate(confident_mask):
                if is_confident:
                    if batch_idx * batch_size + j < len(remaining_unlabeled):
                        global_idx = remaining_unlabeled[batch_idx * batch_size + j]
                        pseudo_indices.append(global_idx)
                        all_pseudo_labeled_indices.add(global_idx)
            batch_idx += 1

        # Update remaining unlabeled indices
        remaining_unlabeled = [idx for idx in remaining_unlabeled if idx not in all_pseudo_labeled_indices]

        # Combine original labeled data with ALL pseudo-labeled data so far
        all_datasets = [original_train_dataset] + all_pseudo_labeled_datasets
        combined_dataset = ConcatDataset(all_datasets)
        combined_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
        print(f" actual dataset number of samples {len(combined_dataset)}")

        # Dispose of the current model and clear GPU memory before reinitializing
        del model
        torch.cuda.empty_cache()

        # Reinitialize model and optimizer for combined training
        model = BaselineModel().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # ensure learning_rate is defined
        

        print(f"Training with {len(combined_dataset)} samples ({len(original_train_dataset)} original labeled + {sum(len(ds) for ds in all_pseudo_labeled_datasets)} pseudo-labeled)")
        print(f"Remaining unlabeled samples: {len(remaining_unlabeled)}")

        best_model_path = f"finetuned_model_iter{iteration}.pth"
        model = train_model(model, num_epochs, optimizer, combined_loader, val_data_loader, criterion, device, best_model_path)

        
        print("start evaluation : ")
        model.eval()
        test_loss, correct_predictions = validate(model, test_data_loader, criterion, device)
        base_accuracy = correct_predictions / len(test_dataset)
        print(f"best model performance at iteratio {iteration} is : {base_accuracy} ")



----------------------------------------------------------------------------------------------------

Pseudo-labeling iteration :  1/3 
 


Generating pseudo-labels:   0%|          | 0/946 [00:00<?, ?it/s]

Generating pseudo-labels: 100%|██████████| 946/946 [04:16<00:00,  3.69it/s]


Generated 16381 pseudo-labeled samples from 30250 with a confidence threshold = 0.95
 actual dataset number of samples 21421
Training with 21421 samples (5040 original labeled + 16381 pseudo-labeled)
Remaining unlabeled samples: 13869
Epoch 1/10


100%|██████████| 40/40 [00:06<00:00,  5.74it/s]


...

100%|██████████| 40/40 [00:13<00:00,  2.97it/s]


Train Loss: 0.0656 Validation Loss: 0.8434 Validation Accuracy: 0.6833
Epoch 9/10


100%|██████████| 40/40 [00:13<00:00,  2.96it/s]


Train Loss: 0.0630 Validation Loss: 0.9826 Validation Accuracy: 0.6389
Epoch 10/10


  0%|          | 0/1020 [00:00<?, ?it/s]